In [1]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Download historische beursgegevens
data = yf.download('AAPL', start='2020-01-01', end='2023-01-01')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
# Definieer een functie voor het genereren van MA crossover signalen
def generate_ma_signals(price, fast_ma, slow_ma, stop_loss):
    # Bereken moving averages
    fast = vbt.MA.run(price, window=fast_ma)
    slow = vbt.MA.run(price, window=slow_ma)

    # Genereer entry signalen (wanneer fast MA boven slow MA kruist)
    entries = fast.ma_above(slow.ma)

    # Genereer exit signalen (wanneer fast MA onder slow MA kruist of de stop loss wordt geraakt)
    exits = fast.ma_below(slow.ma)

    return {
        'entries': entries,
        'exits': exits
    }

# Voorbeeld voor testdoeleinden met enkele parameters
fast_ma = 10
slow_ma = 50
stop_loss = 0.02
signals = generate_ma_signals(data['Close'], fast_ma, slow_ma, stop_loss)

In [4]:
# Definieer portfolio parameters
portfolio_kwargs = {
    'size': 1.0,         # positiegrootte
    'fees': 0.001,       # commissiekosten
    'slippage': 0.001,   # slippage bij in- en uitstappen
    'freq': '1D'         # tijdsfrequentie van de data
}

In [5]:
# Importeer product van itertools
from itertools import product

# Definieer de parameter grid voor de grid search
param_grid = {'fast_ma': np.arange(5, 50, 5), 'slow_ma': np.arange(20, 200, 20),
              'stop_loss': np.arange(0.01, 0.05, 0.01)}

# Genereer alle parameter combinaties
param_combinations = list(
    product(param_grid['fast_ma'], param_grid['slow_ma'], param_grid['stop_loss']))

# Maak lists om de resultaten op te slaan
portfolios = []
param_info = []

# Loop door alle parameter combinaties
for fast_ma, slow_ma, stop_loss in param_combinations:
    # Genereer signalen voor deze parameter combinatie
    signals = generate_ma_signals(data['Close'], fast_ma, slow_ma, stop_loss)

    # Maak portfolio voor deze signalen
    pf = vbt.Portfolio.from_signals(data['Close'], signals['entries'], signals['exits'],
                                    **portfolio_kwargs)

    # Sla portfolio en parameters op
    portfolios.append(pf)
    param_info.append({'fast_ma': fast_ma, 'slow_ma': slow_ma, 'stop_loss': stop_loss})

# Verzamel resultaten in een DataFrame
results = pd.DataFrame(param_info)
# Voeg belangrijke metrics toe
results['total_return'] = [pf.total_return() for pf in portfolios]
results['sharpe_ratio'] = [pf.sharpe_ratio() for pf in portfolios]
results['max_drawdown'] = [pf.max_drawdown() for pf in portfolios]

# Definieer het grid_result object (omzetten naar multi-index)
grid_result = results.set_index(['fast_ma', 'slow_ma', 'stop_loss'])

In [ ]:
# Create a simple class to hold results
class BacktestResult:
    def __init__(self, fast_ma, slow_ma, stop_loss, portfolio):
        self.fast_ma = fast_ma
        self.slow_ma = slow_ma
        self.stop_loss = stop_loss
        self.portfolio = portfolio

        # Calculate metrics
        self.total_return = portfolio.total_return()
        self.sharpe_ratio = portfolio.sharpe_ratio()
        self.max_drawdown = portfolio.max_drawdown()
        self.n_trades = len(portfolio.trades)  # Correct way to get number of trades

    def __repr__(self):
        return f"BacktestResult(fast_ma={self.fast_ma}, slow_ma={self.slow_ma}, stop_loss={self.stop_loss}, return={self.total_return:.2f})"

# Container for all results
all_results = []

# Loop through all parameter combinations
print("Running grid search...")
for fast_ma, slow_ma, stop_loss in param_combinations:
    # Skip invalid combinations
    if fast_ma >= slow_ma:
        continue

    # Calculate moving averages
    fast = vbt.MA.run(data['Close'], window=fast_ma)
    slow = vbt.MA.run(data['Close'], window=slow_ma)

    # Generate entry and exit signals
    entries = fast.ma_above(slow.ma)
    exits = fast.ma_below(slow.ma)

    # Create portfolio
    pf = vbt.Portfolio.from_signals(
        data['Close'],
        entries,
        exits,
        sl_stop=stop_loss,  # Stop loss
        freq='1D',
        fees=0.001,
        slippage=0.001
    )

    # Store result
    result = BacktestResult(fast_ma, slow_ma, stop_loss, pf)
    all_results.append(result)

print(f"Completed {len(all_results)} backtest combinations")

# Convert results to proper DataFrame
data_for_df = []
for result in all_results:
    data_for_df.append({
        'fast_ma': result.fast_ma,
        'slow_ma': result.slow_ma,
        'stop_loss': result.stop_loss,
        'total_return': result.total_return,
        'sharpe_ratio': result.sharpe_ratio,
        'max_drawdown': result.max_drawdown,
        'n_trades': result.n_trades
    })

# Create a clean DataFrame
grid_result = pd.DataFrame(data_for_df)

# Check for NaN values
print(f"Shape of grid_result: {grid_result.shape}")
print(f"NaN values in sharpe_ratio column: {grid_result['sharpe_ratio'].isna().sum()} out of {len(grid_result)}")
print(f"NaN values in total_return column: {grid_result['total_return'].isna().sum()} out of {len(grid_result)}")

# Display sample of the results
print("\nSample of grid_result (first 5 rows):")
print(grid_result.head())

# Find best performing strategy
if not grid_result['sharpe_ratio'].isna().all():
    # Find the best sharpe ratio
    best_idx = grid_result['sharpe_ratio'].idxmax()
    best_params = grid_result.loc[best_idx]

    print("\nBest parameters by Sharpe ratio:")
    print(f"Fast MA: {best_params['fast_ma']}")
    print(f"Slow MA: {best_params['slow_ma']}")
    print(f"Stop Loss: {best_params['stop_loss']}")
    print(f"Total Return: {best_params['total_return']:.4f}")
    print(f"Sharpe Ratio: {best_params['sharpe_ratio']:.4f}")
    print(f"Max Drawdown: {best_params['max_drawdown']:.4f}")
    print(f"Number of Trades: {best_params['n_trades']}")

    # Save the best result for plotting
    best_result = all_results[best_idx]
else:
    print("\nNo valid Sharpe ratios found. Finding best by total return instead.")
    best_idx = grid_result['total_return'].idxmax()
    best_params = grid_result.loc[best_idx]

    print("\nBest parameters by Total Return:")
    print(f"Fast MA: {best_params['fast_ma']}")
    print(f"Slow MA: {best_params['slow_ma']}")
    print(f"Stop Loss: {best_params['stop_loss']}")
    print(f"Total Return: {best_params['total_return']:.4f}")
    print(f"Max Drawdown: {best_params['max_drawdown']:.4f}")
    print(f"Number of Trades: {best_params['n_trades']}")

    # Save the best result for plotting
    best_result = all_results[best_idx]

Running grid search...
Completed 292 backtest combinations
Shape of grid_result: (292, 7)
NaN values in sharpe_ratio column: 0 out of 292
NaN values in total_return column: 0 out of 292

Sample of grid_result (first 5 rows):
   fast_ma  slow_ma  stop_loss  \
0        5       20       0.01   
1        5       20       0.02   
2        5       20       0.03   
3        5       20       0.04   
4        5       40       0.01   

                                        total_return  \
0  ma_window  ma_window  Ticker
5          20    ...   
1  ma_window  ma_window  Ticker
5          20    ...   
2  ma_window  ma_window  Ticker
5          20    ...   
3  ma_window  ma_window  Ticker
5          20    ...   
4  ma_window  ma_window  Ticker
5          40    ...   

                                        sharpe_ratio  \
0  ma_window  ma_window  Ticker
5          20    ...   
1  ma_window  ma_window  Ticker
5          20    ...   
2  ma_window  ma_window  Ticker
5          20    ...   
3  ma_win

In [ ]:
# Create visualizations for grid search analysis
plt.figure(figsize=(20, 16))

# 1. Heatmap of Sharpe ratio by fast_ma and slow_ma (averaging across stop_loss values)
plt.subplot(2, 2, 1)
pivot_sharpe = grid_result.pivot_table(
    values='sharpe_ratio',
    index='fast_ma',
    columns='slow_ma',
    aggfunc='mean'
)
sns.heatmap(pivot_sharpe, annot=True, cmap='viridis', fmt='.2f')
plt.title('Average Sharpe Ratio by MA Parameters')
plt.xlabel('Slow MA')
plt.ylabel('Fast MA')

# 2. Heatmap of returns
plt.subplot(2, 2, 2)
pivot_returns = grid_result.pivot_table(
    values='total_return',
    index='fast_ma',
    columns='slow_ma',
    aggfunc='mean'
)
sns.heatmap(pivot_returns, annot=True, cmap='RdYlGn', fmt='.2f')
plt.title('Average Total Return by MA Parameters')
plt.xlabel('Slow MA')
plt.ylabel('Fast MA')

# 3. Bar chart showing impact of stop loss
plt.subplot(2, 2, 3)
stop_loss_impact = grid_result.groupby('stop_loss')[['sharpe_ratio', 'total_return', 'max_drawdown']].mean()
stop_loss_impact.plot(kind='bar', ax=plt.gca())
plt.title('Impact of Stop Loss on Strategy Performance')
plt.xlabel('Stop Loss')
plt.ylabel('Average Metric Value')
plt.xticks(rotation=0)
plt.legend()

# 4. Scatter plot - Return vs Drawdown with Sharpe ratio as color
plt.subplot(2, 2, 4)
scatter = plt.scatter(
    grid_result['total_return'],
    grid_result['max_drawdown'],
    c=grid_result['sharpe_ratio'],
    cmap='viridis',
    s=100,  # Fixed size for better visibility
    alpha=0.6
)
plt.colorbar(scatter, label='Sharpe Ratio')
plt.title('Risk-Return Trade-off of Different Parameter Sets')
plt.xlabel('Total Return')
plt.ylabel('Maximum Drawdown')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Recreate the best portfolio to visualize performance
best_params = grid_result.iloc[grid_result['sharpe_ratio'].idxmax()]
fast_ma, slow_ma, stop_loss = best_params['fast_ma'], best_params['slow_ma'], best_params['stop_loss']

# Calculate moving averages
fast = vbt.MA.run(data['Close'], window=fast_ma)
slow = vbt.MA.run(data['Close'], window=slow_ma)

# Generate entry and exit signals
entries = fast.ma_above(slow.ma)
exits = fast.ma_below(slow.ma)

# Create portfolio
best_portfolio = vbt.Portfolio.from_signals(
    data['Close'],
    entries,
    exits,
    sl_stop=stop_loss,
    freq='1D',
    fees=0.001,
    slippage=0.001
)

# Plot the equity curve for the best strategy
plt.figure(figsize=(12, 6))
best_portfolio.plot()
plt.title(f'Equity Curve for Best Strategy (Fast MA: {fast_ma}, Slow MA: {slow_ma}, Stop Loss: {stop_loss})')
plt.grid(True)
plt.show()

# Additional plots to understand parameter distributions
plt.figure(figsize=(18, 10))

# 1. Plot distribution of Sharpe ratios
plt.subplot(2, 3, 1)
sns.histplot(grid_result['sharpe_ratio'], kde=True)
plt.axvline(best_params['sharpe_ratio'], color='r', linestyle='--')
plt.title('Distribution of Sharpe Ratios')
plt.xlabel('Sharpe Ratio')

# 2. Plot distribution of returns
plt.subplot(2, 3, 2)
sns.histplot(grid_result['total_return'], kde=True)
plt.axvline(best_params['total_return'], color='r', linestyle='--')
plt.title('Distribution of Total Returns')
plt.xlabel('Total Return')

# 3. Relationship between number of trades and returns
plt.subplot(2, 3, 3)
sns.scatterplot(x='n_trades', y='total_return', data=grid_result)
plt.title('Trade Frequency vs Returns')
plt.xlabel('Number of Trades')
plt.ylabel('Total Return')
plt.grid(True, alpha=0.3)

# 4. Box plot of returns by fast_ma
plt.subplot(2, 3, 4)
sns.boxplot(x='fast_ma', y='total_return', data=grid_result)
plt.title('Returns Distribution by Fast MA')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 5. Box plot of returns by slow_ma
plt.subplot(2, 3, 5)
sns.boxplot(x='slow_ma', y='total_return', data=grid_result)
plt.title('Returns Distribution by Slow MA')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 6. Box plot of returns by stop_loss
plt.subplot(2, 3, 6)
sns.boxplot(x='stop_loss', y='total_return', data=grid_result)
plt.title('Returns Distribution by Stop Loss')
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create a DataFrame to store results
results = []

# Loop through all parameter combinations
for fast_ma, slow_ma, stop_loss in param_combinations:
    # Skip invalid combinations where fast MA is greater than or equal to slow MA
    if fast_ma >= slow_ma:
        continue

    # Calculate moving averages
    fast = vbt.MA.run(data['Close'], window=fast_ma)
    slow = vbt.MA.run(data['Close'], window=slow_ma)

    # Generate entry and exit signals
    entries = fast.ma_above(slow.ma)
    exits = fast.ma_below(slow.ma)

    # Create portfolio
    pf = vbt.Portfolio.from_signals(
        data['Close'],
        entries,
        exits,
        sl_stop=stop_loss,  # Implement stop loss directly
        freq='1D',
        fees=0.001,
        slippage=0.001
    )

    # Calculate metrics
    total_return = pf.total_return()
    sharpe = pf.sharpe_ratio()
    max_dd = pf.max_drawdown()

    # Store results
    results.append({
        'fast_ma': fast_ma,
        'slow_ma': slow_ma,
        'stop_loss': stop_loss,
        'total_return': total_return,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd,
        'n_trades': pf.trades.count()
    })

# Convert results to DataFrame
grid_result = pd.DataFrame(results)

# Print some information about the results
print(f"Total parameter combinations: {len(grid_result)}")
print(f"NaN values in sharpe_ratio: {grid_result['sharpe_ratio'].isna().sum()}")
print(f"NaN values in total_return: {grid_result['total_return'].isna().sum()}")

# Check if we have any valid Sharpe ratios
if grid_result['sharpe_ratio'].notna().any():
    # Sort by Sharpe ratio and get the best parameters
    best_params = grid_result.loc[grid_result['sharpe_ratio'].idxmax()]

    print("\nBest parameters found:")
    print(f"Fast MA: {best_params['fast_ma']}")
    print(f"Slow MA: {best_params['slow_ma']}")
    print(f"Stop Loss: {best_params['stop_loss']}")
    print(f"Sharpe Ratio: {best_params['sharpe_ratio']:.4f}")
    print(f"Total Return: {best_params['total_return']:.4f}")
    print(f"Max Drawdown: {best_params['max_drawdown']:.4f}")
    print(f"Number of Trades: {best_params['n_trades']}")
else:
    print("\nNo valid Sharpe ratios found.")

    # In this case, let's find the best total return instead
    if grid_result['total_return'].notna().any():
        best_return_params = grid_result.loc[grid_result['total_return'].idxmax()]
        print("\nBest parameters based on Total Return:")
        print(f"Fast MA: {best_return_params['fast_ma']}")
        print(f"Slow MA: {best_return_params['slow_ma']}")
        print(f"Stop Loss: {best_return_params['stop_loss']}")
        print(f"Total Return: {best_return_params['total_return']:.4f}")
    else:
        print("No valid results found. Check if any trades were executed.")

In [ ]:
# Print grid shape and check for NaN values
print(f"Shape of grid_result: {grid_result.shape}")
print(f"NaN values in sharpe_ratio column: {grid_result['sharpe_ratio'].isna().sum()} out of {len(grid_result)}")
print(f"Sample of grid_result (first 5 rows):")
print(grid_result.head())
print(f"NaN values in total_return column: {grid_result['total_return'].isna().sum()} out of {len(grid_result)}")

# Function to get portfolio metrics
def get_portfolio_metrics(fast_ma, slow_ma, stop_loss, price_series):
    fast = vbt.MA.run(price_series, window=fast_ma)
    slow = vbt.MA.run(price_series, window=slow_ma)
    entries = fast.ma_above(slow.ma)
    exits = fast.ma_below(slow.ma)
    pf = vbt.Portfolio.from_signals(
        price_series,
        entries,
        exits,
        sl_stop=stop_loss,
        freq='1D',
        fees=0.001,
        slippage=0.001
    )
    return {
        'total_return': pf.total_return(),
        'sharpe_ratio': pf.sharpe_ratio(),
        'max_drawdown': pf.max_drawdown(),
        'trade_count': len(pf.trades)
    }

# Show detailed metrics for first few parameter combinations
print("\nNumber of trades in first few portfolios:")
for i in range(min(5, len(grid_result))):
    row = grid_result.iloc[i]
    params = {'fast_ma': row['fast_ma'], 'slow_ma': row['slow_ma'], 'stop_loss': row['stop_loss']}
    print(f"Portfolio {i}: fast_ma={params['fast_ma']}, slow_ma={params['slow_ma']}, stop_loss={params['stop_loss']}")
    metrics = get_portfolio_metrics(params['fast_ma'], params['slow_ma'], params['stop_loss'], data['Close'])
    print(f"  Total return: {metrics['total_return']}")
    print(f"  Sharpe ratio: {metrics['sharpe_ratio']}")
    print(f"  Max drawdown: {metrics['max_drawdown']}")
    print(f"  Number of trades: {metrics['trade_count']}")
    print("")

# Find and print the best strategy by Sharpe ratio
if grid_result['sharpe_ratio'].notna().any():
    best_idx = grid_result['sharpe_ratio'].idxmax()
    best_params = grid_result.iloc[best_idx]
    print(f"\nBest strategy by Sharpe ratio:")
    print(f"Fast MA: {best_params['fast_ma']}, Slow MA: {best_params['slow_ma']}, Stop Loss: {best_params['stop_loss']}")
    print(f"Total Return: {best_params['total_return']:.4f}")
    print(f"Sharpe Ratio: {best_params['sharpe_ratio']:.4f}")
    print(f"Max Drawdown: {best_params['max_drawdown']:.4f}")
    print(f"Number of Trades: {best_params['n_trades']}")
else:
    print("\nNo valid Sharpe ratios found in the grid results.")

In [ ]:
if len(grid_result) > 0:
    if grid_result['sharpe_ratio'].notna().any():
        sl_to_use = grid_result.loc[grid_result['sharpe_ratio'].astype(float).idxmax(), 'stop_loss']
    else:
        sl_to_use = grid_result['stop_loss'].iloc[0]

    filtered_results = grid_result[grid_result['stop_loss'] == sl_to_use].copy()

    if len(filtered_results) > 0:
        try:
            pivot_sharpe = filtered_results.pivot(index='fast_ma', columns='slow_ma', values='sharpe_ratio')
            pivot_return = filtered_results.pivot(index='fast_ma', columns='slow_ma', values='total_return')

            fig, axes = plt.subplots(1, 2, figsize=(16, 6))

            sns.heatmap(pivot_return, annot=True, cmap='YlGnBu', ax=axes[0])
            axes[0].set_title(f'Total Return by MA Parameters (Stop Loss={sl_to_use})')

            sns.heatmap(pivot_sharpe, annot=True, cmap='YlGnBu', ax=axes[1])
            axes[1].set_title(f'Sharpe Ratio by MA Parameters (Stop Loss={sl_to_use})')

            plt.tight_layout()
            plt.show()

            if grid_result['sharpe_ratio'].notna().any():
                best_params = grid_result.loc[grid_result['sharpe_ratio'].astype(float).idxmax()]

                fast = vbt.MA.run(data['Close'], window=int(best_params['fast_ma']))
                slow = vbt.MA.run(data['Close'], window=int(best_params['slow_ma']))
                entries = fast.ma_above(slow.ma)
                exits = fast.ma_below(slow.ma)

                best_pf = vbt.Portfolio.from_signals(
                    data['Close'],
                    entries,
                    exits,
                    sl_stop=best_params['stop_loss'],
                    freq='1D',
                    fees=0.001,
                    slippage=0.001
                )

                best_pf.plot()
                plt.title('Cumulative Returns for Best Strategy')
                plt.show()
        except Exception as e:
            print(f"Error creating visualizations: {e}")

In [ ]:
# Before finding the best Sharpe ratio index, check if there are any valid values
if grid_result['sharpe_ratio'].notna().any():
    # Get the index of the row with the highest Sharpe ratio (ignoring NaN values)
    best_sharpe_idx = grid_result['sharpe_ratio'].idxmax()

    print(f"fast_ma: {fast_ma}, slow_ma: {slow_ma}, stop_loss: {stop_loss}")
    print(f"Sharpe ratio: {grid_result['sharpe_ratio'].max()}")
    print(f"Totaal rendement: {grid_result.loc[best_sharpe_idx, 'total_return']}")
    print(f"Maximum drawdown: {grid_result.loc[best_sharpe_idx, 'max_drawdown']}")

    # Visualiseer de resultaten - we moeten een aangepaste visualisatie maken  # Unstack de multi-index DataFrame om een heatmap te kunnen maken  # Additional visualization code...
else:
    print("No valid Sharpe ratio found. Check your data and calculations.")
    print(
        "Grid search results may contain only NaN values for Sharpe ratio.")  # You might want to print the grid_result.head() to see what it contains

In [ ]:
# Debug information
print("Shape of grid_result:", grid_result.shape)
print("NaN values in sharpe_ratio column:", grid_result['sharpe_ratio'].isna().sum(), "out of", len(grid_result))
print("Sample of grid_result (first 5 rows):\n", grid_result.head())

# Check if there are any valid total_return values
print("\nNaN values in total_return column:", grid_result['total_return'].isna().sum(), "out of", len(grid_result))

# Check number of trades in each portfolio to see if any trades were executed
print("\nNumber of trades in first few portfolios:")
for i in range(min(5, len(portfolios))):
    params = param_info[i]
    print(f"Portfolio {i}: fast_ma={params['fast_ma']}, slow_ma={params['slow_ma']}, stop_loss={params['stop_loss']}")
    print(f"  Number of trades: {portfolios[i].count()}")
    print(f"  Total return: {portfolios[i].total_return()}")
    print(f"  Sharpe ratio: {portfolios[i].sharpe_ratio()}")